# Visualización de Datos con Python

## Proyecto Final: Creación de un Tablero de Datos Interactivo con Dash

El objetivo de este proyecto es desarrollar un tablero de datos interactivo utilizando la biblioteca [Dash](https://dash.plotly.com/) en Python. Para ello se pone a disposición un conjunto de datos histórico sobre los Juegos Olímpicos modernos para construir visualizaciones significativas y permitir interacciones con el usuario.


### Dataset

El archivo `juegos_olimpicos.csv` contiene información sobre atletas que compitieron en los Juegos Olímpicos modernos desde Atenas 1896 hasta Río de Janeiro 2016. Este dataset ha sido obtenido de [www.sports-reference.com](http://www.sports-reference.com).


### Diccionario de Datos

| Nombre de columna | Descripción |
|-------------------|-------------|
| ID                | Identificador único para cada atleta |
| Name              | Nombre del atleta |
| Sex               | Sexo |
| Age               | Edad |
| Height            | Altura en centímetros |
| Weight            | Peso en kilogramos |
| Team              | Equipo |
| NOC               | Comité Olímpico Nacional (código de tres letras) |
| Games             | Año y temporada de los juegos |
| Season            | Temporada (invierno o verano) |
| City              | Ciudad |
| Sport             | Deporte |
| Event             | Evento |
| Medal             | Medalla (Oro, Plata, Bronce o NaN si no ganó medalla) |


### Requisitos del Dashboard

1. **Gráfico de Barras**:
   - Mostrar los top *K* países por número de medallas.
   - El usuario debe poder seleccionar el número *k* con un menú desplegable para introducir números enteros, el tipo de medalla (*oro*, *plata*, *bronce*) mediante *checkboxes*, y la disciplina mediante otro menú desplegable, el cual debe incluir la opción "Todas" para contabilizar medallas de todas las disciplinas.

2. **Participación por Sexo**:
   - Visualización de la participación de mujeres versus hombres, graficando con un scatter plot, donde cada punto representará la cantidad de hombres en el eje horizontal, y la cantidad de mujeres en el eje vertical, para cada país. Los puntos correspondientes a cada año deberán tener colores distintos.
   - Incluir una barra deslizante para seleccionar un rango de años.
   - Bonus: Incluir una regresión lineal para ajustar los puntos correspondientes a cada año (Ayuda: Usar `seaborn.regplot`).

3. **Representación Geográfica**:
   - Graficar un planisferio que muestre la distribución de número de atletas (de todas las disciplinas) por país.
   - Implementar callbacks para permitir interactividad para filtrar por año mediante un menú desplegable.
   - Ayuda: Pueden usar la función `choropleth` de Plotly

4. **Distribución de Edad por Sexo**:
   - Realizar dos histogrames en un mismo gráfico mostrando la distribución de edad para cada sexo y para cada disciplina.
   - Utilizar callbacks para permitir al usuario seleccionar diferentes deportes mediante un menú desplegable que incluya la opción "Todos", y filtrar por rango de años mediante una barra deslizante.

5. **Bonus (Opcional)**:
   - Añadir características adicionales al dashboard que mejoren la visualización o interactividad. Esto puede incluir gráficos adicionales, filtros avanzados, etc.
   - Si bien es opcional, suma puntos.


### Evaluación

Se valorará especialmente:

1. **Selección y Presentación de Gráficos**:
   - Asegurarse de que los gráficos sean claros y bien etiquetados (tamaño adecuado, ejes, títulos, etc.).

2. **Implementación de Callbacks**:
   - Uso correcto de callbacks para la interactividad del tablero.

3. **Usabilidad**:
   - El tablero debe ser responsive, asegurando una buena visualización en dispositivos móviles.
   - Disposición lógica y estética de los componentes.


### Entregables

- **Notebook**: Debe contener todo el código necesario para generar el dashboard.
- Instrucciones claras para la instalación de paquetes y ejecución del código.
- El profesor debe poder ejecutar todas las celdas del notebook y visualizar el dashboard, ya sea inline o en una pestaña aparte.


### Implmentacion

0. **Carga de librerias, datos y tratamiento de datos**:

In [490]:
# Dash app original

import plotly.io as pio
import dash_bootstrap_components as dbc
import numpy as np

import pandas as pd
import plotly.express as px
from dash import Dash
from dash import dcc
from dash import html
from dash import ctx,callback
from dash.dependencies import Input, Output

stylesheet = [dbc.themes.BOOTSTRAP]

In [491]:
# Carga inicial de datos
competidores = pd.read_csv("juegos_olimpicos.csv",index_col=0, parse_dates=True,date_format='mixed')
competidores.head()

,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
ID,,,,,,,,,,,,,,
1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN


In [492]:
competidores.info()

<class 'pandas.core.frame.DataFrame'>
Index: 271116 entries, 1 to 135571
Data columns (total 14 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Name    271116 non-null  object 
 1   Sex     271116 non-null  object 
 2   Age     261642 non-null  float64
 3   Height  210945 non-null  float64
 4   Weight  208241 non-null  float64
 5   Team    271116 non-null  object 
 6   NOC     271116 non-null  object 
 7   Games   271116 non-null  object 
 8   Year    271116 non-null  int64  
 9   Season  271116 non-null  object 
 10  City    271116 non-null  object 
 11  Sport   271116 non-null  object 
 12  Event   271116 non-null  object 
 13  Medal   39783 non-null   object 
dtypes: float64(3), int64(1), object(10)
memory usage: 31.0+ MB


In [493]:
competidores.describe()

,Age,Height,Weight,Year
count,261642.000000,210945.000000,208241.000000,271116.000000
mean,25.556898,175.338970,70.702393,1978.378480
std,6.393561,10.518462,14.348020,29.877632
min,10.000000,127.000000,25.000000,1896.000000
25%,21.000000,168.000000,60.000000,1960.000000
50%,24.000000,175.000000,70.000000,1988.000000
75%,28.000000,183.000000,79.000000,2002.000000
max,97.000000,226.000000,214.000000,2016.000000


In [494]:
competidores.duplicated(keep='first').unique()

array([False,  True])

In [495]:
#  Listado de los valores duplicados
competidores[competidores.duplicated(keep="first")]

,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
ID,,,,,,,,,,,,,,
704,Dsir Antoine Acket,M,27.0,NaN,NaN,Belgium,BEL,1932 Summer,1932,Summer,Los Angeles,Art Competitions,"Art Competitions Mixed Painting, Unknown Event",NaN
2449,William Truman Aldrich,M,48.0,NaN,NaN,United States,USA,1928 Summer,1928,Summer,Amsterdam,Art Competitions,"Art Competitions Mixed Painting, Drawings And ...",NaN
2449,William Truman Aldrich,M,48.0,NaN,NaN,United States,USA,1928 Summer,1928,Summer,Amsterdam,Art Competitions,"Art Competitions Mixed Painting, Drawings And ...",NaN
2777,Hermann Reinhard Alker,M,43.0,NaN,NaN,Germany,GER,1928 Summer,1928,Summer,Amsterdam,Art Competitions,"Art Competitions Mixed Architecture, Designs F...",NaN
2777,Hermann Reinhard Alker,M,43.0,NaN,NaN,Germany,GER,1928 Summer,1928,Summer,Amsterdam,Art Competitions,"Art Competitions Mixed Architecture, Architect...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135072,Anna Katrina Zinkeisen (-Heseltine),F,46.0,NaN,NaN,Great Britain,GBR,1948 Summer,1948,Summer,London,Art Competitions,"Art Competitions Mixed Painting, Paintings",NaN
135072,Anna Katrina Zinkeisen (-Heseltine),F,46.0,NaN,NaN,Great Britain,GBR,1948 Summer,1948,Summer,London,Art Competitions,"Art Competitions Mixed Painting, Paintings",NaN
135072,Anna Katrina Zinkeisen (-Heseltine),F,46.0,NaN,NaN,Great Britain,GBR,1948 Summer,1948,Summer,London,Art Competitions,"Art Competitions Mixed Painting, Unknown Event",NaN


In [496]:
# Eliminamos los duplicados, que coinciden en toda la fila
competidores.drop_duplicates(keep="first", inplace=True)
competidores.shape

(269731, 14)

In [497]:
# Verificacion de nulos
competidores.isna().sum()

Name           0
Sex            0
Age         9315
Height     58814
Weight     61527
Team           0
NOC            0
Games          0
Year           0
Season         0
City           0
Sport          0
Event          0
Medal     229959
dtype: int64

*Tratamientos de nulos correspondientes*

In [498]:
# Columna Age
#Tratamos de obtener la mejor media posible
competidores['Age'] = competidores['Age'].fillna(competidores.groupby(['Sex', 'Year','Season','Sport','Event'],observed = True)['Age'].transform('mean'))
# Quitamos algunas restricciones para poder disminuir los nulos
competidores['Age'] = competidores['Age'].fillna(competidores.groupby([ 'Year', 'Season','Sport','Event'],observed = True)['Age'].transform('mean'))
# Asignamos valores a todos los na de edad
competidores['Age'] = competidores['Age'].fillna(competidores.groupby([ 'Year', 'Season', 'Sport'],observed = True)['Age'].transform('mean'))

In [499]:
# Columna Height
# Tratamos de obtener la mejor media posible
competidores['Height'] = competidores['Height'].fillna(competidores.groupby(['Sex', 'Year', 'Season', 'Sport', 'Event'], observed = True)['Height'].transform('mean'))
# Quitamos algunas restricciones para poder disminuir los nulos
competidores['Height'] = competidores['Height'].fillna(competidores.groupby(['Sex'], observed = True)['Height'].transform('mean'))

In [500]:
# Columna Weight
# Tratamos de obtener la mejor media posible
competidores['Weight'] = competidores['Weight'].fillna(competidores.groupby(['Sex', 'Year', 'Season', 'Sport', 'Event'], observed=True)['Weight'].transform('mean'))
# Quitamos algunas restricciones para poder disminuir los nulos
competidores['Weight'] = competidores['Weight'].fillna(competidores.groupby(['Sex', 'Year', 'Season', 'Sport'], observed=True)['Weight'].transform('mean'))
competidores['Weight'] = competidores['Weight'].fillna(competidores.groupby(['Sex', 'Year', 'Season'], observed=True)['Weight'].transform('mean'))
competidores['Weight'] = competidores['Weight'].fillna(competidores.groupby(['Sex', 'Year'], observed=True)['Weight'].transform('mean'))
competidores['Weight'] = competidores['Weight'].fillna(competidores.groupby(['Sex'], observed=True)['Weight'].transform('mean'))

In [501]:
# Columna Medal
# Tratamos de obtener la mejor media posible
competidores['Medal'] = competidores['Medal'].fillna(value='Ninguna')

*Transformacion de tipos de datos de columnas*

In [502]:
# Columna Age
competidores['Age'] = np.floor(competidores['Age']).astype('int')

In [503]:
# Columna Team
competidores['Team'] = competidores['Team'].astype('category')

In [504]:
# Columna Year
competidores['Year'] = competidores['Year'].astype('int')

In [505]:
# Columna Season
competidores['Season'] = competidores['Season'].astype('category')

In [506]:
# Columna City
competidores['City'] = competidores['City'].astype('category')

In [507]:
# Columna Event
competidores['Event'] = competidores['Event'].astype('category')

In [508]:
# Columna Medal
competidores['Medal'] = competidores['Medal'].astype('category')

**Creacion del dash**

In [509]:
# stylesheet = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
# external JavaScript files
external_scripts = [
    'https://www.google-analytics.com/analytics.js',
    {'src': 'https://cdn.polyfill.io/v2/polyfill.min.js'},
    {
        'src': 'https://cdnjs.cloudflare.com/ajax/libs/lodash.js/4.17.10/lodash.core.js',
        'integrity': 'sha256-Qqd/EfdABZUcAxjOkMi8eGEivtdTkh3b65xCZL4qAQA=',
        'crossorigin': 'anonymous'
    }
]

# external CSS stylesheets
external_stylesheets = [
    'https://codepen.io/chriddyp/pen/bWLwgP.css',
    {
        'href': 'https://stackpath.bootstrapcdn.com/bootstrap/4.1.3/css/bootstrap.min.css',
        'rel': 'stylesheet',
        'integrity': 'sha384-MCw98/SFnGE8fJT3GXwEOngsV7Zt27NXFoaoApmYm81iuXoPkFOJwJ8ERdknLPMO',
        'crossorigin': 'anonymous'
    }
]


    
app_dash = Dash(__name__, external_scripts=external_scripts,
                external_stylesheets=external_stylesheets, suppress_callback_exceptions=True)


def run_server(self,
               port=8050,
               debug=True,
               threaded=True,
               **flask_run_options):
    self.server.run(port=port, debug=debug, **flask_run_options)

def div_con_tamaño(children=[], font_size: int = 14, id: str = "undefined"):
    return html.Div(children=children, id=id, style={"fontSize": f"{font_size}px"})


app_dash.layout = html.Div(
    
    children=[
        html.Div(
            children=[
                html.H1('Datos de los juegos olimpicos'),
                dcc.Tabs(id="tabs", value='Punto1', children=[
                        dcc.Tab(label='Medallas por paises', value='Punto1'),
                        dcc.Tab(label='Participaciones por genero', value='Punto2'),
                        dcc.Tab(label='Representacion geografica', value='Punto3'),
                        dcc.Tab(label='Cantidad por edad', value='Punto4')                    
                ])
            ]
        ),
        html.Div(id='contenido')        
    ]
)

@app_dash.callback(Output('contenido', 'children'), 
                   [Input('tabs', 'value')]                   
                   )
def render_tabs_content(tab):    
    if (tab == 'Punto1'):        
        return html.Div(
            [
                dbc.Card(
                    [
                        dbc.CardBody(
                            [
                                dbc.Container([
                                    dbc.Row(
                                        [dbc.Col([html.H1("Medallas por paises.")], width=8)], justify='center'),
                                    dbc.Row(
                                        [
                                            dbc.Col([dcc.Markdown('# Medallas:')], width=1),
                                            dbc.Col([dcc.Checklist(id='chk_cat_medalla',options=cat_medallas,value=cat_medallas, inline= False,labelStyle={'padding':'3px'})], width=1),
                                            dbc.Col([dcc.Markdown('# Paises:')], width=1),
                                            dbc.Col([dcc.Dropdown(options=cant_paises, value='5', id='cmb_cant_pais')], width=1),
                                            dbc.Col([dcc.Markdown('# Deporte:')], width=1),
                                            dbc.Col([dcc.Dropdown(options=cat_deporte, value='Todas', id='cmb_deportes'),], width=1),
                                        ], justify='center'),
                                    dbc.Row([
                                        dbc.Col([
                                            dcc.Graph(id='figura1')
                                        ], width=8)
                                    ])
                                ], fluid=True)
                            ]
                        ),
                    ],
                    style={"width": "100%"},
                )                
            ]
        )
    elif(tab == 'Punto2'):    
        return html.Div(
            [
                dbc.Card(
                    [
                        dbc.CardBody(
                            [
                                dbc.Container([
                                    dbc.Row(
                                        [dbc.Col([html.H1("Participantes por sexo de cada pais, segun el año")], width=12)], justify='left'),
                                    dbc.Row(
                                        [
                                            dbc.Col([dcc.Markdown('# Rango años:')], width=3)
                                        ], justify='left'),
                                    dbc.Row(
                                        [
                                            dbc.Col([dcc.RangeSlider(min=año_minimo, max=año_maximo, step=1,  value=[año_minimo, año_maximo], id='rang_Slider_años', updatemode='drag',marks=años, tooltip={
                                                "always_visible": True,
                                                "template": "{value}"
                                            })], width=12)
                                        ], justify='left'),
                                    dbc.Row([
                                        dbc.Col([
                                            dcc.Graph(id='figura2')
                                        ], width=12)
                                    ])
                                ], fluid=True)
                            ]
                        ),
                    ],
                    style={"width": "100%"},
                )
            ]
        )
    elif (tab == 'Punto3'):
        return html.Div(
            [
                dbc.Card(
                    [
                        dbc.CardBody(
                            [
                                dbc.Container([
                                    dbc.Row(
                                        [dbc.Col([html.H1("Representacion geografica")], width=8)], justify='center'),
                                    dbc.Row(
                                        [
                                            dbc.Col(
                                                [dcc.Markdown('# Año:')], width=1),
                                            dbc.Col([dcc.Dropdown(options=años_3, value=año_minimo_3, id='cmb_años3')], width=2)
                                        ], justify='center'),
                                    dbc.Row([
                                        dbc.Col([
                                            dcc.Graph(id='figura3')
                                        ], width=8)
                                    ])
                                ], fluid=True)
                            ]
                        ),
                    ],
                    style={"width": "100%"},
                )
            ]
        )
    elif (tab == 'Punto4'):  
        return html.Div(
            [
                dbc.Card(
                    [
                        dbc.CardBody(
                            [
                                dbc.Container([
                                    dbc.Row(
                                        [dbc.Col([html.H1("Representacion distribucion")], width=12)], justify='center'),
                                    dbc.Row(
                                        [
                                            dbc.Col([dcc.Markdown('# Deporte:')], width=2),
                                            dbc.Col([dcc.Dropdown(options=cat_deporte4, value='Todas', id='cmb_deportes4')], width=2)
                                        ], justify='center'),
                                    dbc.Row(
                                        [
                                            dbc.Col([dcc.Markdown('# Rango años:')], width=4)
                                        ], justify='left'),
                                    dbc.Row(
                                        [
                                            dbc.Col([dcc.Slider(min=año_minimo_4, max=año_maximo_4, step=1,  value=año_maximo_4, id='slider_años', updatemode='drag',marks=años_4, tooltip={
                                                "always_visible": True,
                                                "template": "{value}"
                                            })], width=12)
                                        ], justify='left'),
                                    dbc.Row([
                                        dbc.Col([
                                            dcc.Graph(id='figura4')
                                        ], width=12)
                                    ])
                                ], fluid=True)
                            ]
                        ),
                    ],
                    style={"width": "100%"},
                )
            ]
        )
    else:
        return html.Div([
            html.H6("Seleccione la pestaña que desea ver")
        ])

In [510]:
# Operaciones para punto 1
cat_medallas = competidores['Medal'].unique().tolist()
cat_medallas.remove('Ninguna')
cat_medallas = sorted(cat_medallas)
cat_medallas

cat_deporte = competidores['Sport'].unique().tolist()
cat_deporte.append('Todas')
cat_deporte = sorted(cat_deporte)
cat_deporte


cant_paises = []
i = 1
while (i <= competidores['Team'].nunique()):
    cant_paises.append(str(i))
    i = i+1
@app_dash.callback(Output('figura1', 'figure'),
                   [
                       Input('chk_cat_medalla', 'value'),
                       Input('cmb_cant_pais', 'value'),
                       Input('cmb_deportes', 'value'),
                   ]
                   )
def actualizar_tab_1(cat_medalla,cantidad,deporte,data=competidores):
    data = None
    
    if (len(cat_medalla) != 0) and (cantidad!=None):        
        for m in cat_medalla:
            if (deporte == 'Todas'):
                data = pd.concat(
                    [
                        data,
                        competidores.loc[competidores['Medal'] == m].groupby(['NOC', "Medal"], observed=True).agg(Cantidad=('Medal', 'count')).reset_index().sort_values(by='Cantidad', ascending=False).head(int(cantidad))
                    ])
            
            else:
                data = pd.concat(
                    [
                        data,competidores.loc[((competidores['Medal'] == m) & (competidores['Sport'] == deporte))].groupby(['NOC', "Medal"], observed=True).agg(Cantidad=('Medal', 'count')).reset_index().sort_values(by='Cantidad', ascending=False).head(int(cantidad))
                        
                    ])
        fig = px.bar(data_frame=data, y="NOC", x='Cantidad',
                     color='Medal', height=400, barmode='group', orientation='h')
        fig.update_traces(marker_color="yellow", selector={"name": "Gold"})
        fig.update_traces(marker_color="brown", selector={"name": "Bronze"})
        fig.update_traces(marker_color="silver", selector={"name": "Silver"})
    else:
        fig = px.bar(data_frame=data)
        #fig.update_layout(transition_duration=500)
    return fig

In [511]:
# operaciones Punt 2
#**Participación por Sexo**:
#   - Visualización de la participación de mujeres versus hombres, graficando con un scatter plot, donde cada punto representará la cantidad de hombres en el eje horizontal, y la cantidad de mujeres en el eje vertical, para cada país. Los puntos correspondientes a cada año deberán tener colores distintos.
#   - Incluir una barra deslizante para seleccionar un rango de años.
#   - Bonus: Incluir una regresión lineal para ajustar los puntos correspondientes a cada año (Ayuda: Usar `seaborn.regplot`).

año_minimo=competidores['Year'].min()
año_maximo = competidores['Year'].max()
años = dict()
tengo_1800 = False
tengo_1900 = False
tengo_2000 = False

for y in sorted(competidores['Year'].unique().tolist()):

    if (y < 1900):
        if (not tengo_1800):
            años[str(y)] = str(y)
            tengo_1800 = True
        else:
            años[str(y)] = str(y)[-2:]
    elif (y < 2000):
        if (not tengo_1900):
            años[str(y)] = str(y)
            tengo_1900 = True
        else:
            años[str(y)] = str(y)[-2:]
    else:
        if (not tengo_2000):
            años[str(y)] = str(y)
            tengo_2000 = True
        else:
            if (y == año_maximo):
                años[str(y)] = str(y)
            else:
                años[str(y)] = str(y)[-2:]

@app_dash.callback(Output('figura2', 'figure'),
                   [
                       Input('rang_Slider_años', 'value')
                    ])
def actualizar_tab_2(rango, data_a_procesar=competidores):
    data = None
    px.scatter(data)
    if (len(rango) != 0):
        data = data_a_procesar.loc[:, ['Year', 'NOC', 'Sex']]
        data['Male']=np.where(data['Sex']=='M',1,0)
        data['Female']=np.where(data['Sex']=='F',1,0)
        data = data.loc[((data['Year'] >= rango[0]) & (data['Year'] <= rango[1]))].groupby(['Year', "NOC"], observed=True).agg(
            Female=('Female', 'sum'),
            Male=('Male', 'sum'),
            ).reset_index().sort_values(by='Year', ascending=False)
        data['Year'] = data['Year'].astype('str')
        fig = px.scatter(data, x="Male", y="Female", color="Year", trendline="ols")
    return fig

In [512]:
# Operaciones Punto 3
año_minimo_3=competidores['Year'].min()
año_maximo_3 = competidores['Year'].max()
años_3=sorted(list(competidores['Year'].unique()))
@app_dash.callback(Output('figura3', 'figure'),
                   [
                       Input('cmb_años3', 'value')
])
def actualizar_tab_3(value, data_a_procesar=competidores):
    data = None
    fig = None
    if (value != None):        
        data = data_a_procesar.loc[(data_a_procesar['Year'] == int(value))].groupby(['Year', "NOC"], observed=True).agg(
            Participantes=('NOC', 'count')
        ).reset_index().sort_values(by='Year', ascending=False)
        fig = px.choropleth(data, locations="NOC",
                            color="Participantes",  # lifeExp is a column of gapminder
                            hover_name="NOC",  # column to add to hover information
                            color_continuous_scale=px.colors.sequential.Plasma)
    return fig

In [513]:
# Operaciones Punto 4

cat_deporte4 = competidores['Sport'].unique().tolist()
cat_deporte4.append('Todas')
cat_deporte4 = sorted(cat_deporte4)

año_minimo_4=competidores['Year'].min()
año_maximo_4 = competidores['Year'].max()
años_4 = dict()
tengo_1800=False
tengo_1900=False
tengo_2000 = False

for y in sorted(competidores['Year'].unique().tolist()):
    
    if(y<1900):
        if(not tengo_1800):
            años_4[str(y)] = str(y)
            tengo_1800=True
        else:
            años_4[str(y)] = str(y)[-2:]
    elif(y<2000):
        if(not tengo_1900):
            años_4[str(y)] = str(y)
            tengo_1900=True
        else:
            años_4[str(y)] = str(y)[-2:]
    else:
        if(not tengo_2000):
            años_4[str(y)] = str(y)
            tengo_2000=True
        else:
            if(y==año_maximo_4):
                años_4[str(y)] = str(y)   
            else:
                años_4[str(y)] = str(y)[-2:]
        



@app_dash.callback(Output('figura4', 'figure'),
                   [
                       Input('cmb_deportes4', 'value'),
                       Input('slider_años', 'value')
                    ]
)
def actualizar_tab_4(deporte,anio, data_a_procesar=competidores):
    data = data_a_procesar.loc[:,['Year','Sex','Sport','Age']]  
    if (deporte == 'Todas'):
            data = data.loc[(data['Year']<=anio)].sort_values(by='Year', ascending=True)
    else:
        data = data.loc[((data['Year'] <= anio) & (data['Sport']==deporte))].sort_values(
            by='Year', ascending=True)
        
    fig = px.histogram(data_frame=data,x='Age',color='Sex',barmode="overlay")
    # Reduce opacity to see both histograms
    fig.update_traces(opacity=0.75)
    return fig

In [514]:
if __name__ == '__main__':
    app_dash.run_server(debug=True,port=8051, jupyter_mode="external")

Dash app running on http://127.0.0.1:8051/


c:\CPE\CEPDVDCP\envWindows\Lib\site-packages\statsmodels\regression\linear_model.py:1783: RuntimeWarning:

invalid value encountered in scalar divide

c:\CPE\CEPDVDCP\envWindows\Lib\site-packages\statsmodels\regression\linear_model.py:1783: RuntimeWarning:

invalid value encountered in scalar divide

c:\CPE\CEPDVDCP\envWindows\Lib\site-packages\statsmodels\regression\linear_model.py:1783: RuntimeWarning:

invalid value encountered in scalar divide

